# Lab 2: Episodic Memory Agent

In Lab 0, the agent had no context and forgot everything after each turn.

In Lab 1, the agent had short-term memory using checkpointing, but forgot everything when starting a new session with a different thread_id.

In Lab 2, we add episodic memory. The agent can now extract, store, and retrieve important facts across sessions. This means the agent remembers past conversations even after restarting or using a new thread_id.

The episodic memory system works by extracting key facts from conversations and storing them in a SQLite database. When the agent starts a new session, it searches this database for relevant memories based on the current conversation context and injects them into the conversation.

This allows the agent to answer questions like "What did we discuss yesterday?" or "What's my ticket number?" without needing the user to repeat information.

IMPORTANT:
- Checkpointer = conversation continuity within a thread
- Episodic memory = durable memory across threads and time
- We intentionally keep episodic memory OUTSIDE LangGraph state
  to mirror production architectures.




In [ ]:
# !pip install langchain langchain-openai langgraph langgraph-checkpoint-sqlite
# (We’ll use sqlitedict to build a SQLite-backed store. 
# In production, you’d swap this out for PostgresStore or RedisStore 
# you could replace SQLiteEpisodicStore.search with a method that 
# uses embeddings and a vector database.
# This allows the agent to find similar meaning 
# (e.g., "slow internet" should match a memory about "low speed") 
# even if keywords don't match.)

In [1]:
import json, uuid, sqlite3
from datetime import datetime, timezone
from typing import List, TypedDict, Annotated, Optional

from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langchain_openai import ChatOpenAI

from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

C:\Users\varsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
llm = ChatOpenAI(model="gpt-5.1", temperature=0.2)

In [ ]:
#SQLite episodic memory store
class SQLiteEpisodicStore:
    def __init__(self, db_path="memories.sqlite"):
        self.conn = sqlite3.connect(db_path, check_same_thread=False)
        self.conn.execute("""
            CREATE TABLE IF NOT EXISTS episodic (
                namespace TEXT,
                key TEXT,
                content TEXT,
                timestamp TEXT
            )
        """)
        self.conn.commit()

    def put(self, namespace: str, key: str, value: dict):
        self.conn.execute(
            "INSERT OR REPLACE INTO episodic (namespace, key, content, timestamp) VALUES (?,?,?,?)",
            (namespace, key, json.dumps(value), datetime.now(timezone.utc).isoformat())
        )
        self.conn.commit()

    def get(self, namespace: str, key: str):
        cur = self.conn.execute(
            "SELECT content FROM episodic WHERE namespace=? AND key=?",
            (namespace, key)
        ).fetchone()
        return json.loads(cur[0]) if cur else None

    def search(self, namespace: str, query: str):
        like = f"%{query}%"
        cur = self.conn.execute(
            "SELECT key, content FROM episodic WHERE namespace=? AND content LIKE ?",
            (namespace, like)
        ).fetchall()
        return [(row[0], json.loads(row[1])) for row in cur]

    def clear(self, namespace: str = None):
        """Clear all memories. If namespace is provided, clear only that namespace."""
        if namespace:
            self.conn.execute("DELETE FROM episodic WHERE namespace=?", (namespace,))
        else:
            self.conn.execute("DELETE FROM episodic")
        self.conn.commit()
        return True

episodic_store = SQLiteEpisodicStore()
# Ideallly, namespace = f"crm_support/{customer_id}
namespace = "crm_support/varsha"

In [4]:
# Load ticket DB (similar to previous labs)
ticket_db = json.load(open("data/tickets.json"))

def lookup_ticket_tool(ticket_id: str):
    """Lookup ticket by ID."""
    ticket = ticket_db.get(ticket_id)
    if ticket is None:
        return {"error": f"Ticket {ticket_id} not found."}
    return {"ticket": ticket, "ticket_id": ticket_id}

def create_ticket_tool(customer_name: str, issue: str, device: str = "-", priority: str = "Medium"):
    """Create a new ticket."""
    # Reload ticket_db to ensure we have the latest data
    global ticket_db
    ticket_db = json.load(open("data/tickets.json"))
    
    new_id = str(max([int(k) for k in ticket_db.keys() if k.isdigit()], default=0) + 1)
    ticket_db[new_id] = {
        "status": "New",
        "issue": issue,
        "description": issue,
        "device": device,
        "priority": priority,
        "created_at": datetime.now().strftime("%Y-%m-%d"),
        "last_updated": datetime.now().strftime("%Y-%m-%d"),
        "customer_name": customer_name,
        "notes": [{"timestamp": datetime.now().isoformat(), "author": "customer", "text": issue}]
    }
    json.dump(ticket_db, open("data/tickets.json", "w"), indent=2)
    return {"ticket_id": new_id, "status": "created", "message": f"Ticket {new_id} created successfully"}

def update_ticket_tool(ticket_id: str, note: str, device: str = None):
    """Add a note or update device info to an existing ticket."""
    global ticket_db
    ticket_db = json.load(open("data/tickets.json"))
    
    if ticket_id not in ticket_db:
        return {"error": f"Ticket {ticket_id} not found."}
    
    ticket = ticket_db[ticket_id]
    
    # Add note
    if note:
        ticket["notes"].append({
            "timestamp": datetime.now().isoformat(),
            "author": "customer",
            "text": note
        })
    
    # Update device if provided
    if device and device != "-":
        ticket["device"] = device
        if ticket.get("description") == ticket.get("issue"):
            ticket["description"] = f"{ticket['issue']} Device: {device}"
    
    ticket["last_updated"] = datetime.now().strftime("%Y-%m-%d")

    # Save to file with proper error handling
    try:
        with open("data/tickets.json", "w") as f:
            json.dump(ticket_db, f, indent=2)
    except Exception as save_error:
        print(f"Error saving ticket {ticket_id} to file: {save_error}")
        return {"error": f"Failed to save ticket: {str(save_error)}"}
    return {"ticket_id": ticket_id, "status": "updated"}


TOOLS = [
    {"name": "lookup_ticket", "func": lambda args: lookup_ticket_tool(args["ticket_id"])},
    {"name": "create_ticket", "func": lambda args: create_ticket_tool(**args)},
    {"name": "update_ticket", "func": lambda args: update_ticket_tool(**args)}
]


In [ ]:
def planner_node(state):
    """
    This planner prompt is intentionally verbose to make decision boundaries explicit for learning. 
    Ideally, this would be split into smaller rule-driven nodes.
    """
    latest = state["messages"][-1].content
    planning_instruction = """
You are a CRM support planner with episodic memory.

DECISION PROCESS:
1. FIRST, check if memories contain ticket information (look for "Ticket 123456" or "ticket 123456")
2. If memories contain a ticket ID AND the user is providing additional details (router model, house info, troubleshooting steps), call update_ticket
3. If user explicitly provides a ticket ID, use that
4. If no ticket exists and user describes a NEW issue, call create_ticket


RULES:
1. If the user provides a ticket ID (a 5-10 digit number) or asks for a ticket update/status/details/history, ALWAYS call:

{
  "action": "tool",
  "tool": "lookup_ticket",
  "arguments": { "ticket_id": "..." },
  "response": ""
}

2. If the user provides ADDITIONAL DETAILS about an existing ticket (router model, house details, troubleshooting steps tried, device info), call update_ticket:

Examples of when to call update_ticket:
- Customer says "I've tried moving the router..." (they already have a ticket)
- Customer says "My router is Netgear..." (providing device details)
- Customer says "I have a 2-story house..." (providing house details)
- Customer mentions troubleshooting steps they already tried

{
  "action": "tool",
  "tool": "update_ticket",
  "arguments": { "ticket_id": "...", "note": "customer provided: router model X, tried Y, etc.", "device": "router model if mentioned" },
  "response": ""
}

IMPORTANT: Extract ticket_id from:
1. Memories (look for "Ticket 123456" or "ticket 123456")
2. Conversation history - check for tool results that contain "ticket_id" (e.g., from create_ticket tool result)
3. User explicitly providing ticket ID

If you find a ticket_id in memories or conversation history AND the user is providing additional details, ALWAYS call update_ticket with that ticket_id.

{
  "action": "tool",
  "tool": "update_ticket",
  "arguments": { "ticket_id": "...", "note": "customer provided: router model X, tried Y, etc.", "device": "router model if mentioned" },
  "response": ""
}

You can infer ticket_id from memories if the customer mentions their issue but not the ticket number explicitly.

3. Only call create_ticket if the user clearly describes a NEW ISSUE and provides the REQUIRED fields:
   - customer_name
   - issue

If you DO call create_ticket, arguments must strictly be:

{
  "customer_name": "...",
  "issue": "...",
  "device": "-",
  "priority": "Medium"
}

4. If required fields are missing, DO NOT CALL create_ticket.
   Instead answer directly and ask the user for missing details.

5. Never guess arguments. Never invent fields that the tool does not accept.

6. If the user asks about past conversations, memories, or "what did we discuss", the memory_read node will have already retrieved relevant memories. Use those memories in your response, but still follow rules 1-5 for ticket operations.

Return strictly JSON:

{
  "action": "tool" | "respond",
  "tool": "lookup_ticket" | "create_ticket" | "update_ticket" | "",
  "arguments": {},
  "response": ""
}
"""

    messages = state["messages"] + [
        HumanMessage(content=planning_instruction)
    ]

    out = llm.invoke(messages)
    
    # Extract JSON from response (might be wrapped in markdown)
    content = out.content
    if "```json" in content:
        content = content.split("```json")[1].split("```")[0].strip()
    elif "```" in content:
        content = content.split("```")[1].split("```")[0].strip()
    
       
    return {"messages": [AIMessage(content=content)]}


In [6]:
def tool_node(state):
    """Execute tool based on planner decision."""
    raw = state["messages"][-1].content
    try:
        plan = json.loads(raw)
    except:
        return {"tool_result": None}

    action = plan.get("action")
    tool_name = plan.get("tool")
        
    if action == "tool" and tool_name:
        for t in TOOLS:
            if t["name"] == tool_name:
                result = t["func"](plan["arguments"])
                return {"tool_result": result, "messages": [SystemMessage(content=f"Tool result: {json.dumps(result)}")]}
    
    return {"tool_result": None}


In [ ]:
# NOTE:
# This is a simple lexical recall strategy with keywords search.
# Better approach would be to replace this with:
# - embeddings + vector search
# - or metadata + recency scoring

def memory_read(state):
    """Always retrieve relevant memories based on current conversation context."""
    # Extract recent conversation for search query
    recent_msgs = state["messages"][-6:] if len(state["messages"]) >= 6 else state["messages"]
    query_text = " ".join([m.content for m in recent_msgs if hasattr(m, 'content')])
    
    # Search memory store - try multiple search terms for better recall
    all_results = []
    query_lower = query_text.lower()
    
    # Extract meaningful words from query (more words, lower threshold)
    words = query_text.split()
    search_terms = [w.strip('.,!?;:') for w in words if len(w.strip('.,!?;:')) > 2]  # Words > 2 chars
    
    # Add common keywords if user asks about tickets/issues/problems
    if any(word in query_lower for word in ["ticket", "issue", "problem", "status", "what", "remember", "was", "my", "more"]):
        search_terms.extend(["ticket", "issue", "problem", "customer", "ID", "troubleshoot"])
    
    # Search with each term
    for term in search_terms[:10]:  # Use up to 10 search terms
        results = episodic_store.search(namespace, term)
        all_results.extend(results)
    
    
    # Also try searching for the full query (for better context matching)
    if len(query_text) > 10:
        full_results = episodic_store.search(namespace, query_text[:50])  # First 50 chars
        all_results.extend(full_results)
    
    # Deduplicate by key
    seen_keys = set()
    unique_results = []
    for key, value in all_results:
        if key not in seen_keys:
            seen_keys.add(key)
            unique_results.append((key, value))
    
    if not unique_results:
        return {"messages": []}
    
    # Format memories for injection into context
    memory_text = "Relevant past memories from previous sessions:\n"
    memory_text += "\n".join([f"- {r[1].get('content', '')}" for r in unique_results[:5]])  # Top 5
    
    return {"messages": [SystemMessage(content=memory_text)]}


In [ ]:
# NOTE:
# currently we extract memory after every response, memory_write should be gated by salience scoring or explicit triggers.

EXTRACT_PROMPT = """
Extract a concise episodic memory from the conversation. ALWAYS include ticket IDs if mentioned or created.

Focus on:
- Customer issues reported (be specific)
- Ticket IDs created or mentioned (CRITICAL - always include if present)
- Troubleshooting steps taken
- Customer details (name, device model, etc.)
- Important context that should persist across sessions

Output JSON: {"key":"<short_id>","content":"<summary>"}
or {"content":""} if nothing worth remembering.

IMPORTANT: If a ticket was created, the ticket ID MUST be included in the memory.

Example: {"key":"internet_drop_issue","content":"Customer Varsha reported internet drops every 20 minutes after 10 PM. Ticket 293445 created. Router model: Archer-X600. Suggested router power cycle."}
"""

def memory_write(state):
    """Extract and store episodic memories after each interaction."""
    # Get last few messages (user + agent)
    recent = state["messages"][-6:] if len(state["messages"]) >= 6 else state["messages"]
    conversation = "\n".join([f"{type(m).__name__}: {m.content}" for m in recent])
    
    # Extract memory using LLM
    response = llm.invoke([
        SystemMessage(content=EXTRACT_PROMPT),
        HumanMessage(content=conversation)
    ])
    
    try:
        # Try to parse JSON from response
        content = response.content
        # Extract JSON if wrapped in markdown
        if "```json" in content:
            content = content.split("```json")[1].split("```")[0].strip()
        elif "```" in content:
            content = content.split("```")[1].split("```")[0].strip()
        
        mem = json.loads(content)
        if mem.get("content") and len(mem["content"].strip()) > 10:  # Only store non-trivial memories
            key = mem.get("key") or f"memory_{uuid.uuid4().hex[:8]}"
            episodic_store.put(namespace, key, {"content": mem["content"]})
    except json.JSONDecodeError as e:
        print(f"Memory extraction failed - JSON parse error: {e}")
        print(f"Response content: {response.content[:200]}...")
    except Exception as e:
        print(f"Memory extraction failed - Error: {type(e).__name__}: {e}")
        print(f"Response content: {response.content[:200] if hasattr(response, 'content') else 'N/A'}...")
    
    return {}


In [9]:
RESPONSE_SYSTEM = """You are a CRM support agent with episodic memory. 

CRITICAL: If past memories are provided, you MUST use them to answer questions.

Rules:
- If memories contain a ticket ID and the user asks about their ticket, use that ticket ID from memory
- If memories mention the customer's issue, router model, or troubleshooting steps, reference them
- NEVER ask for information that is already in the provided memories
- If tool_result contains ticket data, summarize it clearly
- If memories and tool_result both have ticket info, prefer the tool_result (it's current) but acknowledge the memory

Keep responses concise and helpful. Show that you remember past conversations."""

def response_node(state):
    """Generate response using conversation history + retrieved memories + tool results."""
    msgs = state["messages"].copy()  # Already includes memory_read SystemMessage if memories found
    
    # Add tool result context if present
    tool_result = state.get("tool_result")
    if tool_result:
        tool_context = SystemMessage(content=f"Tool execution result: {json.dumps(tool_result)}")
        msgs.append(tool_context)
    
    # Prepend system instruction
    if not any(isinstance(m, SystemMessage) and "CRM support agent" in m.content for m in msgs):
        msgs = [SystemMessage(content=RESPONSE_SYSTEM)] + msgs
    
    reply = llm.invoke(msgs)
    return {"messages": [reply]}


In [10]:
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    tool_result: Optional[dict]  


In [11]:
# Create SQLite checkpointer for graph state persistence
checkpointer = MemorySaver()

graph = StateGraph(AgentState)

graph.add_node("memory_read", memory_read)  # Always read memories first
graph.add_node("planner", planner_node)
graph.add_node("tool", tool_node)
graph.add_node("respond", response_node)
graph.add_node("memory_write", memory_write)  # Always write after response

graph.set_entry_point("memory_read")  # Start with memory retrieval
graph.add_edge("memory_read", "planner")  # Then plan action
graph.add_edge("planner", "tool")  # Execute tool if needed
graph.add_edge("tool", "respond")  # Generate response
graph.add_edge("respond", "memory_write")  # Store memories
graph.add_edge("memory_write", END)

# Compile with SQLite checkpointer for state persistence
app = graph.compile(checkpointer=checkpointer)


In [12]:
def run_chat(inputs, thread_id="default_thread"):
    """Run chat with persistent state across calls using checkpointer."""
    for text in inputs:
        print(f"\n🧑USER: {text}")
        # Invoke with thread_id - checkpointer will persist state automatically
        out = app.invoke(
            {"messages": [HumanMessage(content=text)]},
            {"configurable": {"thread_id": thread_id}}
        )
        print(f"🤖 AGENT: {out['messages'][-1].content}")
        print("=" * 60)
    return out

In [ ]:
# Demo: Session 1 - Create ticket and establish context
print("=" * 60)
print("SESSION 1 (Day 1): Customer reports issue and creates ticket")
print("=" * 60)
session1 = run_chat([
    "Hi, I'm Varsha. I'm having WiFi dead zones in my house - I can't get internet in my bedroom and kitchen. Can you create a support ticket?",
    "I've tried moving the router to different locations, but the signal is still weak in those rooms.",
    "My router is a Netgear Nighthawk R7000, and I have a 2-story house with the router on the ground floor.",
    "Can you tell me my ticket number?"
], thread_id="session1")

print("\n" + "=" * 60)
print("Session 1 complete. Current Issue details and previous chats should be stored in episodic memory.")
print("=" * 60)


SESSION 1 (Day 1): Customer reports issue and creates ticket

🧑USER: Hi, I'm Varsha. I'm having WiFi dead zones in my house - I can't get internet in my bedroom and kitchen. Can you create a support ticket?
🤖 AGENT: I’ve created a support ticket for you.

- Ticket ID: **998883**
- Issue: WiFi dead zones at home (no coverage in bedroom and kitchen)
- Status: **Created**

Next steps:
1. Our support team will review your ticket and may contact you for preferred callback time or additional details about your home layout.
2. In the meantime, if you know your router model or where it’s placed (e.g., living room, near a wall, in a cabinet), tell me and I can suggest quick changes to improve coverage while you wait.

🧑USER: I've tried moving the router to different locations, but the signal is still weak in those rooms.
🤖 AGENT: Thanks for the update, Varsha. I’ve added this to your ticket **998883**: you’ve already tried moving the router but the bedroom and kitchen still have weak signal.

A

### Session 2: Next day - Episodic memory in action
**Key test**: The agent should remember:
- The specific issue (WiFi dead zones in bedroom and kitchen)
- The router model (Netgear Nighthawk R7000)
- What troubleshooting was already tried (moving router locations)
- The ticket number (without asking)
- All devices affected

This demonstrates **cross-session continuity** - the agent maintains context even after a restart.


In [13]:
# Demo: Session 2 - Next day, agent should remember EVERYTHING
print("\n" + "=" * 60)
print("SESSION 2 (Day 2): Customer returns - Episodic Memory Test!")
print("=" * 60)
print("Note: This is a FRESH session (messages cleared), but memories persist in SQLite.")
print("=" * 60)
session2 = run_chat([
    # Test 1: Agent should know ticket number from memory (no need to ask)
    "What's the status of my ticket?",
    
    # Test 2: Agent should remember what was already tried (avoid repetition)
    "Is there any other troubleshooting we could try?",
    
    # Test 4: Agent should remember specific details (router model, devices)
    "What's my router model again?"

], thread_id="session2")  # Fresh session (new thread_id), but episodic memories persist



SESSION 2 (Day 2): Customer returns - Episodic Memory Test!
Note: This is a FRESH session (messages cleared), but memories persist in SQLite.

🧑USER: What's the status of my ticket?
🤖 AGENT: You’re asking about your existing Wi‑Fi dead zone ticket for your bedroom and kitchen in your 2‑story home (Netgear Nighthawk R7000) — that’s ticket **#998883**.

Here’s the current status:

- **Status:** In Progress  
- **Priority:** High (it was escalated because multiple rooms are affected)  
- **Last updated:** 2025-12-14  

**Latest notes from the technical team:**
- They’ve confirmed recommendations for your setup:
  1. Reposition the R7000 to a **more central spot on the ground floor**, as open and elevated as possible.
  2. Use the **2.4 GHz band** for devices in the bedroom and kitchen for better range.
  3. If coverage is still poor after that, they **recommend adding a mesh system or Wi‑Fi extenders**, ideally placing a node/extender roughly between the router and the dead zones (for ex

### Session 3: One week later - Long-term memory persistence
This demonstrates that episodic memory persists across **multiple sessions over time**. The agent should still remember all the context from Sessions 1 and 2.


In [ ]:
# Demo: Session 3 - One week later, memory still intact
print("\n" + "=" * 60)
print("SESSION 3 (Day 7): One week later - Long-term Memory Test!")
print("=" * 60)
print("The agent should still remember everything from Day 1 and Day 2.")
print("=" * 60)
session3 = run_chat([
    # Test: Agent should remember the original issue from a week ago
    "I had a WiFi problem last week. Can you remind me what it was?",
    
    # Test: Agent should recall the ticket without being told
    "What's the latest on my ticket?",
    
    # Test: Agent should remember specific technical details
    "What router do I have again?",
    
    # Test: Agent should remember what troubleshooting was done
    "Did I already try moving the router?What all troubleshooting we already did?",
    
], thread_id="session3")  # Another fresh session (new thread_id), episodic memories still persist


### View Stored Episodic Memories

Let's inspect what memories have been stored in the SQLite database:


In [14]:
# Query all stored memories for the customer
import sqlite3
from datetime import datetime, timezone

def view_all_memories(namespace="crm_support/varsha"):
    """Display all stored episodic memories."""
    conn = sqlite3.connect("memories.sqlite")
    cursor = conn.execute(
        "SELECT key, content, timestamp FROM episodic WHERE namespace=? ORDER BY timestamp DESC",
        (namespace,)
    )
    
    memories = cursor.fetchall()
    conn.close()
    
    if not memories:
        print("No memories stored yet.")
        return
    
    print(f"Found {len(memories)} stored memories for {namespace}:\n")
    print("=" * 80)
    
    for i, (key, content_json, timestamp) in enumerate(memories, 1):
        try:
            content_dict = json.loads(content_json)
            memory_content = content_dict.get('content', '')
        except:
            memory_content = content_json
        
        print(f"\n[{i}] Key: {key}")
        print(f"    Timestamp: {timestamp}")
        print(f"    Content: {memory_content}")
        print("-" * 80)
    
    return memories

# View all memories
all_memories = view_all_memories()


Found 7 stored memories for crm_support/varsha:


[1] Key: wifi_dead_zones_varsha
    Timestamp: 2025-12-13T08:44:47.766711+00:00
    Content: Customer Varsha has WiFi dead zones in the bedroom and kitchen of a 2‑story home, tracked under ticket #998883. Router model is Netgear Nighthawk R7000 on the ground floor. Multiple troubleshooting steps suggested, including optimizing 2.4/5 GHz settings, channel selection, router placement, antenna angles, firmware updates, and interference checks. Issue likely due to coverage/placement; mesh or extender may be needed.
--------------------------------------------------------------------------------

[2] Key: wifi_dead_zones_varsha_998883
    Timestamp: 2025-12-13T08:44:42.387448+00:00
    Content: Customer Varsha has persistent WiFi dead zones in bedroom and kitchen of a 2‑story home using a Netgear Nighthawk R7000 on the ground floor. Ticket 998883 is In Progress with High priority. Customer has already tried moving the router. Support and tec

### Search for Specific Memories

You can also search for memories by keyword:


In [15]:
# Search for memories containing specific keywords
from pathlib import Path

def search_memories(query, namespace="crm_support/varsha"):
    """Search memories by keyword."""
    results = episodic_store.search(namespace, query)
    
    if not results:
        print(f"No memories found containing '{query}'")
        return
    
    print(f"Found {len(results)} memories containing '{query}':\n")
    for key, value in results:
        content = value.get('content', '') if isinstance(value, dict) else str(value)
        print(f"  • {key}: {content}\n")

# Example searches
print("Searching for 'internet' related memories:")
search_memories("internet")

print("\n" + "="*80 + "\n")
print("Searching for 'router' related memories:")
search_memories("router")

print("\n" + "="*80 + "\n")
print(f"Database location: {Path('memories.sqlite').resolve()}")
print("You can also open this SQLite file with any SQLite browser to inspect the data.")


Searching for 'internet' related memories:
Found 1 memories containing 'internet':

  • varsha_wifi_dead_zones: Customer Varsha reported WiFi dead zones at home with no internet coverage in the bedroom and kitchen. Support ticket 998883 created with medium priority. No specific router/device model provided yet; issue likely related to home coverage and router placement. Suggested interim troubleshooting based on router location and home layout while support team follows up.



Searching for 'router' related memories:
Found 7 memories containing 'router':

  • varsha_wifi_dead_zones: Customer Varsha reported WiFi dead zones at home with no internet coverage in the bedroom and kitchen. Support ticket 998883 created with medium priority. No specific router/device model provided yet; issue likely related to home coverage and router placement. Suggested interim troubleshooting based on router location and home layout while support team follows up.

  • wifi_dead_zones_varsha: Customer Varsh